# Example notebook for the Rigol DG 1062 instrument

In [1]:
import time 

from qcodes.instrument_drivers.rigol.DG1062 import DG1062

Instantiate the driver 

In [2]:
gd = DG1062("gd", "TCPIP0::169.254.187.99::INSTR")

Connected to: Rigol Technologies DG1062Z (serial:DG1ZA195006397, firmware:03.01.12) in 0.18s


## Basic usage

Accessing the channels

In [3]:
gd.channels[0]
# Or...
gd.ch1

<DG1062Channel: gd_ch1 of DG1062: gd>

Trun the output for channel 1 to "on"

In [4]:
gd.channels[0].state(1)
# This is idential to 
gd.ch1.state(1)

With `apply` we can check which waveform is being generated now, for example on channel 1

In [5]:
gd.channels[0].current_waveform()

{'waveform': 'SIN', 'freq': 1000.0, 'ampl': 1.0, 'offset': 0.0, 'phase': 0.0}

We can also change the waveform 

In [8]:
gd.channels[0].apply(waveform="SIN", freq=2000, ampl=0.5, offset=0.0, phase=0.0)

Change individual settings like so: 

In [9]:
gd.channels[0].offset(0.1)

This works for every setting, except waveform, which is read-only

In [10]:
gd.channels[0].waveform()

'SIN'

In [11]:
try: 
    gd.channels[0].waveform("SIN")
except NotImplementedError: 
    print("We cannot set a waveform like this ")

We cannot set a waveform like this 


We can however do this: 

In [12]:
gd.channels[0].sin(freq=1E3, ampl=1.0, offset=0, phase=0)

To find out which arguments are applicable to a waveform: 

Find out which waveforms are available

In [3]:
print(gd.waveforms)

['HARM', 'NOIS', 'RAMP', 'SIN', 'SQU', 'TRI', 'USER', 'DC', 'ARB']


## Setting the impedance 

In [4]:
gd.channels[1].impedance(50)

In [5]:
gd.channels[1].impedance()

50.0

In [14]:
gd.channels[1].impedance("HighZ") 

Alternatively, we can do 

```python
gd.channels[1].impedance("INF")
```

In [7]:
gd.channels[1].impedance()

'HighZ'

## Sync commands 

In [8]:
gd.channels[0].sync()

'ON'

In [9]:
gd.channels[0].sync("OFF") 

Alternativly we can do 

```python
gd.channels[0].sync(0) 
```

In [11]:
gd.channels[0].sync()

'OFF'

In [12]:
gd.channels[0].sync(1)

Alternativly we can do

```python
gd.channels[0].sync("ON")
```

In [15]:
gd.channels[0].sync()

'ON'

## Burst commands 

### Internally triggered burst 

In [16]:
# Interal triggering only works if the trigger source is manual 
gd.channels[0].burst.source("MAN")

In [21]:
# The number of cycles is infinite 
gd.channels[0].burst.mode("INF")

If we want a finite number of cycles: 

```python
gd.channels[0].burst.mode("TRIG")
gd.channels[0].burst.ncycles(10000)
```

Setting a period for each cycle: 

```python
gd.channels[0].burst.period(1E-3)
```

In [22]:
# Put channel 1 in burst mode 
gd.channels[0].burst.on(1)
# Turn on the channel. For some reason, if we turn on the channel 
# immediately after turning on the burst, we trigger immediately. 
time.sleep(0.1)
gd.channels[0].state(1)

In [23]:
# Finally, trigger the AWG
gd.channels[0].burst.trigger()

### extranally triggered burst 

In [24]:
gd.channels[0].burst.source("EXT")

### Setting the idle level

In [25]:
# Set the idle level to First PoinT
gd.channels[0].burst.idle("FPT")

In [28]:
# We can also give a number 
gd.channels[0].burst.idle(0)